##PNG

In [1]:
!apt-get -q install poppler-utils
!pip -q install pdf2image

'apt-get' は、内部コマンドまたは外部コマンド、
操作可能なプログラムまたはバッチ ファイルとして認識されていません。


In [5]:
import requests
from pdf2image import convert_from_path
from pathlib import Path
import numpy as np
import cv2
import os,glob,random
import matplotlib.pyplot as plt

url = "https://www.sci.yamagata-u.ac.jp/wasan/data/sakuma-0035.pdf"
book = requests.get(url)
wasan_pdf =open("sakuma-0035.pdf","wb")
for chunk in book.iter_content(100000) :
  wasan_pdf.write(chunk)
wasan_pdf.close()
!rm -r /content/image/
!mkdir /content/image/
pdf_path = Path("sakuma-0035.pdf")
img_path=Path("image/")
images = convert_from_path(pdf_path, output_folder=img_path,fmt='png',output_file="sakuma-0035-")

!rm -r /content/image-data/
!mkdir /content/image-data/
save_dir = "image-data/"
files=glob.glob("image/*.png")
files.sort()
n=0
allImages = []
for file in files:
    img = cv2.imread(file,cv2.IMREAD_GRAYSCALE)
    allImages.append(img)
    img = img=255*(img < 128 )
    outfile = save_dir+"sakuma-0035-00"+str(n).zfill(2)+".png"
    cv2.imwrite(outfile, img)
    n+=1

files=glob.glob("image-data/*.png")
files.sort()
files.pop(0)
files.pop(-1)

PDFInfoNotInstalledError: Unable to get page count. Is poppler installed and in PATH?

##noise

In [ ]:
!rm -r /content/image-median-3/
!mkdir /content/image-median-3/

In [ ]:
import cv2

n=0
for file in files:
    img = cv2.imread(file,cv2.IMREAD_GRAYSCALE)
    # メディアンフィルタ
    median_filter = cv2.medianBlur(img,   # 入力画像
                                  3      # カーネルサイズ（奇数）
                                  )
    img = median_filter=255*(img < 128 )
    outfile = "image-median-3/"+"sakuma-0035-00"+str(n).zfill(2)+".png"
    cv2.imwrite(outfile, img)
    n+=1

files=glob.glob("image-median-3/*.png")
files.sort()

##blob

In [ ]:
# Standard imports
import numpy as np;
import sys

# original code : http://scikit-image.org/docs/dev/auto_examples/features_detection/plot_blob.html
# other version : https://github.com/TheLaueLab/blob-detection

from math import sqrt
from skimage import data
from skimage.transform import rescale
from skimage.feature import blob_dog, blob_log, blob_doh
from skimage.color import rgb2gray
import matplotlib.pyplot as plt
import os

def imcrop(img, bbox):
    x1, y1, x2, y2 = bbox
    if x1 < 0 or y1 < 0 or x2 > img.shape[1] or y2 > img.shape[0]:
        img, x1, x2, y1, y2 = pad_img_to_fit_bbox(img, x1, x2, y1, y2)
    return img[y1:y2, x1:x2]

def pad_img_to_fit_bbox(img, x1, x2, y1, y2):
    img = cv2.copyMakeBorder(img, - min(0, y1), max(y2 - img.shape[0], 0),-min(0, x1), max(x2 - img.shape[1], 0),cv2.BORDER_REPLICATE)
    y2 += -min(0, y1)
    y1 += -min(0, y1)
    x2 += -min(0, x1)
    x1 += -min(0, x1)
    return img, x1, x2, y1, y2

def overlappingKeyPoints(p,kpList,th):
    returnList=[p]

    #put all superposed to p
    i=0
    while i<len(returnList):
        x=returnList[i]
        for q in kpList:
            if (cv2.KeyPoint_overlap(x,q)>th or cv2.KeyPoint_overlap(q,x)>th )  and not q in returnList:
                #print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@apending with i "+str(i))
                returnList.append(q)
        i+=1
    return returnList

def refineKeypoints(kpList,th):
    returnList=[]
    checkedList=[]
    for kp in kpList:
        if not kp in checkedList:
            overlapping=overlappingKeyPoints(kp,kpList,th)
            if len(overlapping)>1:
                #print("found "+str(len(overlapping)))
                #merge overlapping overlappingKeyPoints
                maxX=overlapping[0].pt[0]
                minX=overlapping[0].pt[0]
                maxY=overlapping[0].pt[1]
                minY=overlapping[0].pt[1]
                for kp2 in overlapping:
                    checkedList.append(kp2)
                    #print ("point "+str(kp2.pt[0])+" , "+str(kp2.pt[1])+" size "+str(kp2.size))
                    if(kp2.pt[0]<minX):minX=kp2.pt[0]
                    if(kp2.pt[0]>maxX):maxX=kp2.pt[0]
                    if(kp2.pt[1]<minY):minY=kp2.pt[1]
                    if(kp2.pt[1]>maxY):maxY=kp2.pt[1]
                #now create new keypoints
                #newP=cv2.KeyPoint(minX+(maxX-minX)/2,minY+(maxY-minY)/2,max((maxX-minX),((maxY-minY))))
                #print(" creating point "+str(newP.pt[0])+" , "+str(newP.pt[1])+" size "+str(newP.size))
                returnList.append(cv2.KeyPoint( minX+(maxX-minX)/2,minY+(maxY-minY)/2,2*max((maxX-minX),((maxY-minY)))))
            else:
                checkedList.append(kp)
                returnList.append(kp)
            #print("returnList lenght  " +str(len(returnList))+" checked list lenght "+str(len(checkedList)))

    return returnList



def SimpleBlobDetector(argv,im):

    # Read image
    #im = cv2.imread(argv[1], cv2.IMREAD_GRAYSCALE)

    # Setup SimpleBlobDetector parameters.
    params = cv2.SimpleBlobDetector_Params()

    # Change thresholds
    params.minThreshold = 10
    params.maxThreshold = 200

    # Filter by Area.
    params.filterByArea = True
    params.minArea = 200
    params.maxArea = 1500

    # Filter by Circularity
    params.filterByCircularity = True
    params.minCircularity = 0.2

    # Filter by Convexity
    params.filterByConvexity = False
    params.minConvexity = 0.25
    params.maxConvexity = 0.75

    # Filter by Inertia
    params.filterByInertia = False
    params.minInertiaRatio = 0.01

    if len(argv)>4: params.minArea=int(min_sigma)


    # Create a detector with the parameters
    ver = (cv2.__version__).split('.')
    if int(ver[0]) < 3 :
        detector = cv2.SimpleBlobDetector(params)
    else :
        detector = cv2.SimpleBlobDetector_create(params)


    # Detect blobs.
    keypoints = detector.detect(im)

    #now refine keypoints so overlapping blobs are merged
    overlapThreshold=0.001
    #print ("number of keypoints "+str(len(keypoints)))

    refinedKeypoints=refineKeypoints(keypoints,overlapThreshold)
    #print ("number of keypoints "+str(len(refinedKeypoints)))

    #return keypoints
    return refinedKeypoints

def detect_mser(image, name_file, output_file):
    # MSERオブジェクトを作成
    mser = cv2.MSER_create()

    # MSERを実行してブロブを検出
    regions, _ = mser.detectRegions(image)

    # ブロブを囲む矩形を描画するためのコピーを作成
    mser_image = image.copy()

    # 検出したブロブに対して領域をリサイズして保存
    for i, region in enumerate(regions):
        x, y, w, h = cv2.boundingRect(region)
        resized_blob = image[y:y+h, x:x+w]  # ブロブの領域を切り取り
        cv2.imwrite(f"/content/image-PATCHES/"+name_file+"/kanji"+str(i).zfill(4)+".png", resized_blob)

        # 検出したブロブを描画
        cv2.rectangle(mser_image, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imwrite(output_file, mser_image)
    return 0


In [ ]:
# Blod detector tester function
# blob:https://colab.research.google.com/04f004a1-8937-43d9-9a9a-476b578fd821

for file in files: # use image-median-3
    input_file = file
    output_file = input_file.split("/")[-1]
    name_file = output_file.split(".")[0]
    output_file = "image-blob/"+name_file+"_blob.png"
    blob_pattern = 3
    min_sigma = 25  # Defaults=1
    max_sigma = 30  # Defaults=50
    num_sigma = 25  # between `min_sigma` and `max_sigma`
                    # Defaults=10
    threshold = .01
    overlap = .3  # 0 < overlap < 1
    mask_file = input_file.split(".")[0]+"_mask_"+str(n).zfill(4)+".png" # maskファイル

    list=[input_file, output_file, blob_pattern, min_sigma, max_sigma, num_sigma, threshold, overlap, mask_file]

    image_gray=cv2.imread(input_file, cv2.IMREAD_GRAYSCALE)
    image=image_gray
    image_black=(255-image_gray)

    #let's create a text output file too
    positionsOutputFile="kanjiPATH/"+name_file+"_POSITIONS.txt"
    kanjiPATHOutputFile="kanjiPATH/"+name_file+"_PATH.txt"
    print("positions file "+str(positionsOutputFile))
    print("output file "+output_file)
    #open it too
    posF = open(positionsOutputFile, "a")
    kanjiPATH = open(kanjiPATHOutputFile, "a")

    maskOutputFile = "image-maskOutputFiles/" + mask_file
    maskCentroidsFile = "image-maskCentroidsFiles/" + mask_file

    #switch between the different types of blob detector
    if int(blob_pattern)==0:
        keypoints=SimpleBlobDetector(list,image)
        #print ("number of keypoints outside "+str(len(keypoints)))

    elif int(blob_pattern)==1:
        #parameters: http://scikit-image.org/docs/dev/api/skimage.feature.html#skimage.feature.blob_log
        #example!, other parameters exist
        if len(list)>4: min_s=int(min_sigma)
        if len(list)>5: max_s=int(max_sigma)
        if len(list)>6: num_s=int(num_sigma)
        if len(list)>7: thres=float(threshold)
        if len(list)>8: over=float(overlap)
        #print("starting laplacion of gaussians detector with parameters "+str(min_s)+" "+str(over))
        blob_List = blob_log(image_black, min_sigma=min_s, max_sigma=max_s, num_sigma=num_s, threshold=thres, overlap=over)
        # Compute radii in the 3rd column.
        blob_List[:, 2] = blob_List[:, 2] * sqrt(2)
    elif int(blob_pattern)==2:
        #parameters: http://scikit-image.org/docs/dev/api/skimage.feature.html#skimage.feature.blob_dog
        if len(list)>4: min_s=int(min_sigma)
        if len(list)>5: max_s=int(max_sigma)
        if len(list)>6: sigma=float(num_sigma)
        if len(list)>7: thres=float(threshold)
        if len(list)>8: over=float(overlap)
        blob_List = blob_dog(image_black, min_sigma=min_s, max_sigma=max_s, sigma_ratio=sigma, threshold=thres, overlap=over)
        blob_List[:, 2] = blob_List[:, 2] * sqrt(2)
    elif int(blob_pattern)==3:
        #parameters: http://scikit-image.org/docs/dev/api/skimage.feature.html#skimage.feature.blob_doh
        if len(list)>4: min_s=int(min_sigma)
        if len(list)>5: max_s=int(max_sigma)
        if len(list)>6: num_s=int(num_sigma)
        if len(list)>7: thres=float(threshold)
        if len(list)>8: over=float(overlap)
        blob_List = blob_doh(image, min_sigma=min_s, max_sigma=max_s, num_sigma=num_s, threshold=thres, overlap=over)
    elif int(blob_pattern)==4:
        # mser
        pass
    else:
        print("Blob detector type not supported: "+blob_pattern)
        sys.exit(-1)

    #Now write the results to file
    if int(blob_pattern)==1 or int(blob_pattern)==2 or int(blob_pattern)==3:
        for blob in blob_List:
            y, x, r = blob
            cv2.rectangle(image,(int(x-r), int(y-r)), (int(x+r), int(y+r)), (0, 0, 255), 1)
            #cv2.circle(image,(int(x),int(y)),int(r),(0,0,255))
        cv2.imwrite(output_file,image)
        Y, X = image.shape[0], image.shape[1]
        outputMask=np.ones((Y,X),np.uint8)
        outputMaskCentroids = outputMask.copy()
        i=0
        #print("Number of Blobs "+str(len(blob_List)))
        for blob in blob_List:
            #print("opencv detector painting blob "+str(i))
            y, x, r = blob
            #画像の端のFDを除去する
            if 0.05*Y < y < 0.95*Y and 0.05*X < x < 0.95*X:
                outputMask[int(y-r):int(y+r),int(x-r):int(x+r)]=255
                outputMaskCentroids[int(y-r*0.7):int(y+r*0.7),int(x-r*0.7):int(x+r*0.7)]=255
                i=i+1

        invertOutputMask=255-outputMask
        invertOutputMaskCentroids=255-outputMaskCentroids
        cv2.imwrite(maskOutputFile,invertOutputMask)
        cv2.imwrite(maskCentroidsFile,invertOutputMaskCentroids)

    elif int(blob_pattern)==4: pass # mserは下記に書く

    else:
        # must be 0 as we already controled other cases
        # Draw detected blobs as red circles.
        # cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS ensures the size of the circle corresponds to the size of blob
        #im_with_keypoints = cv2.drawKeypoints(image, keypoints, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        # cv2.imwrite("sambombita.jpg",im_with_keypoints)

        for kp in keypoints:
            x = kp.pt[0]
            y = kp.pt[1]
            r = kp.size
            cv2.rectangle(image,(int(x-r), int(y-r)), (int(x+r), int(y+r)), (0, 0, 255), 1)
            #cv2.circle(image,(int(x),int(y)),int(r),(0,0,255))
        cv2.imwrite(output_file,image)

    #now crop each kanji image
    #sys.exit(0)
    #print("now Cropping")
    try:
        os.mkdir("image-PATCHES/"+name_file)
    except:
        #print("An exception occurred Probably directory already existed")
        pass
    margin=5
    scale_percent = 5000 # percent of original size

    if int(blob_pattern)==1 or int(blob_pattern)==2 or int(blob_pattern)==3:
        i=0
        image=cv2.imread(input_file, cv2.IMREAD_GRAYSCALE)
        #print("Number of Blobs "+str(len(blob_List)))
        print(str(len(blob_List)))
        for blob in blob_List:
            #print("Cropping blob "+str(i))
            # print(blob)
            y, x, r = blob
            if 0.05*Y < y < 0.95*Y and 0.05*X < x < 0.95*X:
                posF.write("kanji"+str(int(i))+".png"+" "+str(int(y))+" "+str(int(x))+" "+str(r)+"\n")
                kanjiPATH.write("image-PATCHES/"+name_file+"/kanji"+str(i).zfill(4)+".png"+"\n")

                currentKanjiImage=imcrop(image, (int(x-r-margin), int(y-r-margin), int(x+r+margin), int(y+r+margin) ) )
                newImageName="image-PATCHES/"+name_file+"/kanji"+str(i).zfill(4)+".png"
                cv2.imwrite(newImageName,currentKanjiImage)
                width = int(currentKanjiImage.shape[1] * scale_percent / 100)
                height = int(currentKanjiImage.shape[0] * scale_percent / 100)
                #width = int(800)
                #height = int(593)
                dim = (width, height)

                # resize image
                resized = cv2.resize(currentKanjiImage, dim, interpolation = cv2.INTER_NEAREST)

                #negative image
                negImage=(255-resized)

                cv2.imwrite(newImageName,negImage)

                i=i+1

    elif  int(blob_pattern)==4:
        detect_mser(image, name_file, output_file)

    else:
        i=0
        image=cv2.imread(input_file, cv2.IMREAD_GRAYSCALE)
        #print("Number of Blobs "+str(len(keypoints)))
        print(str(len(keypoints)))
        for kp in keypoints:
            #print("Cropping simple blob  "+str(i))
            x = kp.pt[0]
            y = kp.pt[1]
            r = kp.size

            currentKanjiImage=imcrop(image, (int(x-r-margin), int(y-r-margin), int(x+r+margin), int(y+r+margin) ) )
            # name = output_file.split("/")[-1]
            newImageName="image-PATCHES/"+name_file+"_PATCHES/kanji"+str(i).zfill(4)+".png"

            posF.write("kanji"+str(int(i))+" "+str(int(y))+" "+str(int(x))+" "+str(r)+"\n")

            cv2.imwrite(newImageName,currentKanjiImage)
            width = int(currentKanjiImage.shape[1] * scale_percent / 100)
            height = int(currentKanjiImage.shape[0] * scale_percent / 100)
            #width = int(800)
            #height = int(593)
            dim = (width, height)

            # resize image
            resized = cv2.resize(currentKanjiImage, dim, interpolation = cv2.INTER_NEAREST)

            #negative image
            negImage=(255-resized)

            cv2.imwrite(newImageName,negImage)

            i=i+1

kanjiPATH.close()
posF.close()


print("finish")
